In [1]:
!pip3 install pandas==1.3.1
!pip3 install numpy==1.21.1

In [2]:
#for colab users and python<3.8
!pip3 install pickle5
import pickle5 as pickle

#for python >= 3.8 RUN below command and comment above command
#import pickle

In [3]:
import pandas as pd
import numpy as np

In [4]:
# Accessing the research papers and data

with open('C:/Users/Alexis/Desktop/SistemaQABert/metadata_sample2.pickle', 'rb') as f:
  df_metadata = pickle.load(f)
with open('C:/Users/Alexis/Desktop/SistemaQABert/json_pdf_sample2.pickle', 'rb') as f:
  df_pdf = pickle.load(f)
with open('C:/Users/Alexis/Desktop/SistemaQABert/json_pmc_sample2.pickle', 'rb') as f:
  df_pmc = pickle.load(f)

Muestreo y almacenamiento del conjunto de datos del curso


In [5]:
#Merging the metadata and json pdf data
#sha column of metadata and paper_id of json_pdf data are same identifier for each record
df_merged = pd.merge(df_metadata,df_pdf,left_on='sha',right_on='paper_id',how='left').drop('paper_id',axis=1)

In [6]:
df_merged.head(1)
#abstract_x is the abstract from metadata and abstract_y is the abstract from json_pdf

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract_x,publish_time,...,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,abstract_y,body_text
0,bliggtp4,543c26359460283ea08a8ee1ada672deeab82863,Medline; PMC,Systemic steroids may enhance recovery from lo...,10.1002/alr.22848,PMC8427029,34132052,no-cc,NaN,2021-06-16,...,Int Forum Allergy Rhinol,NaN,NaN,NaN,document_parses/pdf_json/543c26359460283ea08a8...,document_parses/pmc_json/PMC8427029.xml.json,https://doi.org/10.1002/alr.22848; https://www...,235448953.0,,Chemosensory dysfunction (CD) is among the mos...


In [6]:
#Lets merge the json_pmc data to the merged data too
df_merged = pd.merge(df_merged,df_pmc,left_on='pmcid',right_on='paper_id',how='left').drop('paper_id',axis=1)
df_merged.shape

(50000, 22)

Limpieza y preprocesamiento de datos

In [7]:
df_merged[df_merged.abstract_x != df_merged.abstract_y].shape

(45475, 22)

In [8]:
# check metadata abstract column to see if null values exist
df_merged.abstract_x.isnull().sum(),(df_merged.abstract_x == '').sum()

(6924, 0)

In [9]:
# Check pdf_json abstract to see if null values exist
df_merged.abstract_y.isnull().sum(),(df_merged.abstract_y == '').sum()

(3078, 13618)

Dado que el abstract_x de los metadatos es más fiable, lo utilizaremos pero sólo lo rellenaremos con el texto abstract_y cuando el valor de abstract_x sea nulo


In [10]:
# Convert all columns to string and then replace abstract_y values
#df = df.astype(str)
df_merged["abstract_y"] = df_merged["abstract_y"].astype(str) 
df_merged['abstract_y'] = np.where(df_merged['abstract_y'].map(len) > 50, df_merged['abstract_y'], "na")

In [11]:
df_merged[df_merged['abstract_y'].apply(lambda x: len(str(x)) <= 10)]

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract_x,publish_time,...,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,abstract_y,body_text_x,body_text_y
1,fb6myqtr,dd4ca66c448e89e4c841f29daf40a356ef67babe; f4d5...,Medline; PMC,Towards biomarkers for outcomes after pancreat...,10.1136/bmjopen-2020-039560,PMC7747584,33334830,cc-by-nc,INTRODUCTION: Ageing-related processes such as...,2020-12-17,...,NaN,NaN,NaN,document_parses/pdf_json/dd4ca66c448e89e4c841f...,document_parses/pmc_json/PMC7747584.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33334830/;...,229316922.0,na,NaN,The incidence of pancreatic cancer is increasi...
4,y6y235hw,303efcb8c5b2c65281962e16039caf0a18acda73; 89d8...,Elsevier; Medline; PMC,Steroid-associated osteonecrosis: Epidemiology...,10.1016/j.jot.2014.12.002,PMC5982361,30035041,cc-by-nc-nd,Steroid-associated osteonecrosis (SAON) is a c...,2015-01-13,...,NaN,NaN,NaN,document_parses/pdf_json/303efcb8c5b2c65281962...,document_parses/pmc_json/PMC5982361.xml.json,https://www.sciencedirect.com/science/article/...,49895240.0,na,NaN,Osteonecrosis (ON) is a disabling clinical dis...
5,wbkwp5aj,0f7125bdd3fabee194a3fbbed3893ad51644da03,Medline; PMC,First Surveillance of Violence against Women d...,10.3390/ijerph18073801,PMC8038697,33917305,cc-by,Violence against women emerges with tragic reg...,2021-04-06,...,NaN,NaN,NaN,document_parses/pdf_json/0f7125bdd3fabee194a3f...,document_parses/pmc_json/PMC8038697.xml.json,https://doi.org/10.3390/ijerph18073801; https:...,233208208.0,na,Episodes of violence against women emerge with...,Episodes of violence against women emerge with...
9,7w7bi56o,21e8f9dd0cfdccde3e8a49e711af9bc28cef958b,PMC,Definition of HCV Reactivation [Letter],10.2147/imcrj.s342905,PMC8547488,34712063,cc-by-nc,NaN,2021-10-22,...,NaN,NaN,NaN,document_parses/pdf_json/21e8f9dd0cfdccde3e8a4...,document_parses/pmc_json/PMC8547488.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,NaN,na,We have read with great interest the paper by ...,We have read with great interest the paper by ...
11,ap1ier4j,9ccc485ed46f4509d0f294555a38137e076436ee,Medline; PMC,SARS‐CoV2 virions in PM(10) pollutants. May fu...,10.1002/jmv.26590,PMC7675441,33038013,no-cc,The animated debate about the possibility that...,2020-10-30,...,NaN,NaN,NaN,document_parses/pdf_json/9ccc485ed46f4509d0f29...,document_parses/pmc_json/PMC7675441.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33038013/;...,222255277.0,na,The animated debate about the possibility that...,The animated debate about the possibility that...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49986,rds8o5z4,defd90d30189ee3f111999fba52835d0af6e80b4,Elsevier; Medline; PMC,"The Past, Present, and Future of Simulation-ba...",10.1016/j.cpem.2016.05.005,PMC7106197,32288645,no-cc,The pediatric emergency medicine (PEM) environ...,2016-05-26,...,NaN,NaN,NaN,document_parses/pdf_json/defd90d30189ee3f11199...,document_parses/pmc_json/PMC7106197.xml.json,https://www.sciencedirect.com/science/article/...,77417855.0,na,T he pediatric emergency medicine (PEM) enviro...,Life-threatening events and cardiopulmonary ar...
49990,v4r2ek6a,60c371d44690cafde9637a72d502f93477632052,Medline; PMC; WHO,Coronavirus Disease (COVID-19) in Italy: Analy...,10.3389/fmed.2020.00140,PMC7161343,32328496,cc-by,NaN,2020-04-09,...,NaN,NaN,NaN,document_parses/pdf_json/60c371d44690cafde9637...,document_parses/pmc_json/PMC7161343.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/32328496/;...,215516460.0,na,"Until March 28, 2020, there were ∼90,000 confi...",The lethality rate is determined as follows: t...
49993,7y17ucne,36e62dae00d797cf29939309594dc550d5b5dd98; d23b...,Elsevier; PMC,The role of teledentistry in Oral Surgery duri...,10.1016/j.adoms.2020.100005,PMC7813496,NaN,els-covid,Objectives Evaluation of patient experience of...,2021-01-18,...,NaN,NaN,NaN,document_parses/pdf_json/36e62dae00d797cf29939...,NaN,https://api.elsevier.com/content/arti

In [12]:
# sustituir abstract_x (columna de metadatos) por abstract_y (pdf_json) valor donde abstract_x es null
df_merged.loc[df_merged.abstract_x.isnull() & (df_merged.abstract_y != 'na'),'abstract_x'] = df_merged[df_merged.abstract_x.isnull() & (df_merged.abstract_y != 'na')].abstract_y

In [13]:
# ¿Tenemos algún valor abstracto nulo restante? Ya no. Esto es bueno.
# Los valores nulos se han reducido que es lo que esperábamos.
df_merged.abstract_x.isnull().sum()


5740

In [14]:
# Deshagámonos de la columna pdf_json abstract y cambiemos el nombre de la columna metadata abstract
df_merged.rename(columns = {'abstract_x' : 'abstract'}, inplace = True)
df_merged.drop('abstract_y',axis=1,inplace = True)
df_merged.columns

Index(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal', 'mag_id',
       'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files',
       'url', 's2_id', 'body_text_x', 'body_text_y'],
      dtype='object')

In [15]:
# Esto se espera porque el texto del cuerpo viene de las carpetas pdf y pmc
(df_merged.body_text_x != df_merged.body_text_y).sum()

50000

In [16]:
df_merged.body_text_x.isnull().sum(),(df_merged.body_text_y == '').sum()

(3078, 0)

In [17]:
# Esto se espera porque hay menos papeles en json_pmc
df_merged.body_text_y.isnull().sum()

10596

In [18]:
# body_text_x es pdf_json. body_text_y viene de pmc_json
# Cuando está disponible utilizamos el texto del archivo pmc confiando en la calidad de la declaración
df_merged.body_text_x.isnull().sum(),(df_merged.body_text_y.isnull()).sum()

(3078, 10596)

In [19]:
# Hay ~13k filas en las que body_text_x es null pero body_text_y no es null# Hay ~13k filas en las que body_text_x es null pero body_text_y no es null
df_merged.loc[df_merged.body_text_x.isnull() & df_merged.body_text_y.notnull()]

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,body_text_x,body_text_y
1,fb6myqtr,dd4ca66c448e89e4c841f29daf40a356ef67babe; f4d5...,Medline; PMC,Towards biomarkers for outcomes after pancreat...,10.1136/bmjopen-2020-039560,PMC7747584,33334830,cc-by-nc,INTRODUCTION: Ageing-related processes such as...,2020-12-17,...,BMJ Open,NaN,NaN,NaN,document_parses/pdf_json/dd4ca66c448e89e4c841f...,document_parses/pmc_json/PMC7747584.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33334830/;...,229316922.0,NaN,The incidence of pancreatic cancer is increasi...
4,y6y235hw,303efcb8c5b2c65281962e16039caf0a18acda73; 89d8...,Elsevier; Medline; PMC,Steroid-associated osteonecrosis: Epidemiology...,10.1016/j.jot.2014.12.002,PMC5982361,30035041,cc-by-nc-nd,Steroid-associated osteonecrosis (SAON) is a c...,2015-01-13,...,J Orthop Translat,NaN,NaN,NaN,document_parses/pdf_json/303efcb8c5b2c65281962...,document_parses/pmc_json/PMC5982361.xml.json,https://www.sciencedirect.com/science/article/...,49895240.0,NaN,Osteonecrosis (ON) is a disabling clinical dis...
14,nz4lyeg1,ba9640511aa108c1be7af50c1fdc3384f22b9f3d; 7992...,Elsevier; Medline; PMC,Searching potential antiviral candidates for t...,10.1016/j.heliyon.2020.e04640,PMC7409764,32802981,no-cc,"In the present work, the succinic acid (SA), L...",2020-08-06,...,Heliyon,NaN,NaN,NaN,document_parses/pdf_json/ba9640511aa108c1be7af...,document_parses/pmc_json/PMC7409764.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/32802981/;...,221012129.0,NaN,December 2019 winsted the awfully outbreak of ...
19,abrweljg,c14160d7c97c5cf7b4d0bee2a31da3d838e5cb4d; 21f6...,Elsevier; Medline; PMC,A Comparison of Two Methods for Retrieving ICD...,10.1016/j.jbi.2011.01.005,PMC3440000,21262390,green-oa,OBJECTIVE: Most existing controlled terminolog...,2011-04-01,...,Journal of Biomedical Informatics,NaN,NaN,NaN,document_parses/pdf_json/c14160d7c97c5cf7b4d0b...,document_parses/pmc_json/PMC3440000.xml.json,http://europepmc.org/articles/pmc2655821?pdf=r...,1536708.0,NaN,Most existing controlled terminologies can be ...
36,i3jsvtwp,6596d704c6f6c3072661fc7f58203cfbb43985fa; 5d12...,Medline; PMC,Excess mortality in the United States in the 2...,10.1073/pnas.2024850118,PMC8072323,33846260,cc-by-nc-nd,We use three indexes to identify how age-speci...,2021-04-20,...,Proc Natl Acad Sci U S A,NaN,NaN,NaN,document_parses/pdf_json/6596d704c6f6c3072661f...,document_parses/pmc_json/PMC8072323.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33846260/;...,233222199.0,NaN,Fig. 1A shows the ratio of US death rates to t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49930,vcz6z51b,71d9dc2d598ce67bded3a1339daf5592f6a69933; 7f4e...,BioRxiv; Medline; PMC; WHO,Subtle differences in the pathogenicity of SAR...,10.1101/2021.05.07.443115,PMC8357058,34382034,cc0,The emergence of several SARS-CoV-2 variants h...,2021-05-07,...,bioRxiv,NaN,NaN,NaN,document_parses/pdf_json/71d9dc2d598ce67bded3a...,document_parses/pmc_json/PMC8357058.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/34382034/;...,234350149.0,NaN,"As the COVID-19 pandemic continues, so does th..."
49931,6i7g9uzj,23aa14f686cc1c522054d3b394dd5258daf16077; a640...,Medline; PMC,Single-cell analysis of two severe COVID-19 pa...,10.1038/s41467-020-17834-w,PMC7413381,32764665,cc-by,Several studies show that the immunosuppressiv...,2020-08-06,...,Nat Commun,NaN,NaN,NaN,document_parses/pdf_json/23aa14f686cc1c522054d...,document_parses/pmc_json/PMC7413381.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/32764665/;...,220981673.0,NaN,"As of May 1, 2020, the WHO has reported 224,17..."
49932,7pgglbk0,7b7b3dd035adb07fd4cad7acdf9cd491d79c2f17; bfb7...,Elsevier; Medline; PMC,Demystifying pulmonary vascular complications ...,10.1016/j.thromres.2020.06.043,PMC7319924,32620455,no-cc,• Small pulmonary vessel changes are a unique ...,2020-06-27,...,Thromb Res,NaN,NaN,NaN,d

In [20]:
# Confiamos en que el texto de la carpeta pmc sea de mayor calidad ya que contiene el texto completo. 
# Por lo tanto, vamos a reemplazar con body_text_x con body_text_y donde body_text_y existe
df_merged.loc[df_merged.body_text_y.notnull(),'body_text_x'] = df_merged.loc[df_merged.body_text_y.notnull(), 'body_text_y']

In [21]:
# Deshagámonos de la columna pdf_pmc body text y cambiemos el nombre de la columna body textdf_merged.rename(columns = {'body_text_x' : 'body_text'}, inplace = True)
df_merged.rename(columns = {'body_text_x' : 'body_text'}, inplace = True)
df_merged.drop('body_text_y',axis=1,inplace = True)
df_merged.columns

Index(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal', 'mag_id',
       'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files',
       'url', 's2_id', 'body_text'],
      dtype='object')

In [22]:
# Los valores nulos del cuerpo del texto ahora han disminuido. 
df_merged.body_text.isnull().sum()

192

In [23]:
df_merged.columns

Index(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal', 'mag_id',
       'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files',
       'url', 's2_id', 'body_text'],
      dtype='object')

In [24]:
df_final = df_merged[['sha', 'title', 'abstract', 'publish_time', 'authors', 'url', 'body_text']]

In [25]:
df_final.head()

,sha,title,abstract,publish_time,authors,url,body_text
0,689dc11605e42381fcc7a5e46d29a9bd26a17526,Backbone reconstruction in temporal networks f...,Many complex systems are characterized by time...,2019-10-15,"Surano, Francesco Vincenzo; Bongiorno, Christi...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,"In the past few decades, network science has e..."
1,dd4ca66c448e89e4c841f29daf40a356ef67babe; f4d5...,Towards biomarkers for outcomes after pancreat...,INTRODUCTION: Ageing-related processes such as...,2020-12-17,"Henze, Larissa; Walter, Uwe; Murua Escobar, Hu...",https://www.ncbi.nlm.nih.gov/pubmed/33334830/;...,The incidence of pancreatic cancer is increasi...
2,9ceebe88ad70fbd159e889c9071d4e85684d7138,Steroid treatment in patients with acute respi...,PURPOSE: Although the most recent systematic r...,2021-11-10,"Yoshihiro, Shodai; Hongo, Takashi; Ohki, Shing...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,Acute respiratory distress syndrome (ARDS) is ...
3,e7e2de0dc748780122aa7be286cb300138bb431e,A global analysis on the effect of temperature...,We performed a global analysis with data from ...,2020-10-06,"Rahman, Mizanur; Islam, Mahmuda; Shimanto, Meh...",https://doi.org/10.1007/s10668-020-01028-x; ht...,COVID-19 has become a global concern since the...
4,303efcb8c5b2c65281962e16039caf0a18acda73; 89d8...,Steroid-associated osteonecrosis: Epidemiology...,Steroid-associated osteonecrosis (SAON) is a c...,2015-01-13,"Xie, Xin-Hui; Wang, Xin-Luan; Yang, Hui-Lin; Z...",https://www.sciencedirect.com/science/article/...,Osteonecrosis (ON) is a disabling clinical dis...


In [26]:
df_final = df_final.dropna(axis=0,subset=['body_text', 'title'])
df_final.isnull().sum()

sha                0
title              0
abstract        5710
publish_time       0
authors          454
url                0
body_text          0
dtype: int64

In [27]:
df_final.shape

(49805, 7)

# GUARDANDO DATOS FILTRADOS

In [28]:
df_final.to_csv('FINAL_CORD_DATA2.csv', index=False)

 # Análisis Exploratorio de Datos - Data Analysis (EDA) 

In [29]:
!pip install seaborn==0.11.1

In [1]:
import seaborn as sns
sns.set_theme(style="darkgrid")
import calendar

Revisión de los datos finales 

In [3]:
#Inspect data
def inspect_data(data):
    return pd.DataFrame(
        {
            "Data Type": data.dtypes,
            "No of levels": data.apply(lambda x:len(x.unique()), axis=0),
            "levels": data.apply(lambda x:x.unique(), axis=0),
            "qtd_null": data.apply(lambda x: x.isnull().sum(), axis=0),
            "pct_null": data.apply(lambda x: x.isnull().sum() * 100 / len(x))
        }
    )

In [4]:
inspect_data(df_final)

NameError: name 'df_final' is not defined

In [34]:
df_final['publish_time'] = pd.to_datetime(df_final['publish_time'])

In [35]:
df_final['publish_year'] =  df_final['publish_time'].dt.year
df_final['publish_month'] =  df_final['publish_time'].dt.month
df_final['publish_day'] =  df_final['publish_time'].dt.day

In [36]:
df_final['publish_year'].nunique()

55

In [37]:
df_final['publish_year'].min()

1955

In [ ]:
df_final['publish_year'].max()

In [ ]:
# Nº de artículos publicados en los últimos años
sns.countplot(x="publish_year", data=df_final[df_final['publish_year'].isin([2018, 2019,2020,2021, 2022])])

In [ ]:
#Distribución por meses de los trabajos publicados
sns.countplot(x="publish_month", data=df_final)

In [ ]:
df_final['publish_month'] = df_final['publish_month'].apply(lambda x : calendar.month_abbr[x])

In [ ]:
sns.countplot(x="publish_month", data=df_final)

In [ ]:
#Comprobación del % de trabajos publicados en tiempos recientes frente a años anteriores
df_final['in_covid_time'] = np.where(df_final['publish_year'].isin([2019,2020,2021]), 1, 0)
sns.countplot(df_final['in_covid_time'])

**Revisión de nubes del palabras**

In [ ]:
!pip3 install wordcloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS 
import re
import string
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def plot_word_cloud(wordcloud):
    plt.figure(figsize=(10, 10))
    plt.imshow(wordcloud) 
    plt.axis("off");

In [ ]:
def remove_special_charac_and_punctuations(text) :
# define el patrón a mantener
  pattern = r'[^a-zA-z0-9\'\s]' 
  txt = re.sub(pattern, '', text)
  txt = ''.join([c for c in txt if c not in string.punctuation])
  return  txt

In [ ]:
def concat_all_sentences(sents):
   all_tokens = ''
   for text in sents:
        tx = str(text).lower().strip()
        tx = " ".join(tx.split())
        all_tokens += tx + " "
   return all_tokens  

In [ ]:
cleaned_titles = df_final.loc[df_final.title.notnull(), 'title'].apply(remove_special_charac_and_punctuations)

In [ ]:
cleaned_abstracts = df_final.loc[df_final.abstract.notnull(), 'abstract'].apply(remove_special_charac_and_punctuations)

In [ ]:
title_tokens = concat_all_sentences(cleaned_titles)

In [ ]:
abstract_tokens = concat_all_sentences(cleaned_abstracts)

In [ ]:
title_tokens

In [ ]:
abstract_tokens

In [ ]:
# Observando la nube de palabras de los tokens del título
title_wordcloud = WordCloud(width = 500, height = 500,random_state=10).generate(title_tokens)
plot_word_cloud(title_wordcloud)

## Modelización de temas

In [ ]:
!pip install gensim==3.8.3

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel

In [ ]:
abstracts = cleaned_abstracts.values.tolist()

In [ ]:
# simple_preprocess of genism: elimina los caracteres especiales, los números y los convierte en minúsculas
gensim.utils.simple_preprocess("12 happy today @# '' '\n'", deacc=True) 

In [ ]:
def token_builder(sentences) :
   for sent in sentences:     
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  


In [ ]:
# Construcción de tokens abstractos usando token_builder
abstract_tokens = list(token_builder(abstracts))

In [ ]:
# Construcción de los modelos de bigramas y trigramas
bigram = gensim.models.Phrases(abstract_tokens, min_count=3, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[abstract_tokens], threshold=50)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
!pip install nltk

In [ ]:
import nltk as nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

In [ ]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

In [ ]:
def process_docs(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    # nlp = spacy.load('en', disable=['parser', 'ner'])
    nlp = spacy.load("en_core_web_sm")
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # Eliminar las palabras de parada una vez más después de la lematización   
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out] 
    return texts_out

In [ ]:
abstracts_data = process_docs(abstract_tokens)

In [ ]:
# Creación de un Diccionario de datos
id2word = corpora.Dictionary(abstracts_data)

In [ ]:
print(id2word)

In [ ]:
print(id2word.token2id)

In [ ]:
# Creación del Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in abstracts_data]

In [ ]:
# Construcción del modelo LDA 
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=2, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

In [ ]:
print(lda_model.print_topics())

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.colors as mcolors

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

cloud = WordCloud(stopwords=stop_words,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_model.show_topics(formatted=False)

for i in range(0,2):
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')
    plt.show()

In [ ]:
topics = lda_model.show_topics(formatted=False)

In [ ]:
topics